<a href="https://colab.research.google.com/github/OOJNIM/23fall_LB-C/blob/main/class2023Fall_1103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Introduction to FastAPI for Server-Side Applications**

FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.

---
### **1. Setting up FastAPI**

To get started with FastAPI, you need to install it:

```python
!pip install fastapi[all]
!pip install uvicorn
```

---
### **2. Creating a Simple API with FastAPI**

Let's build a simple API that returns a greeting message.

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello, World!"}
```

---
### **3. Running the FastAPI App**

To run the FastAPI app, you need `uvicorn`. In a local environment, you'd use the terminal, but in Google Colab, you'd have to find another way because `uvicorn` runs indefinitely. Here's how you can do it:

```python
# =====================================
# !pip install nest-asyncio
# import nest_asyncio
# import uvicorn

# nest_asyncio.apply()

# def run():
#     uvicorn.run(app, host="0.0.0.0", port=8000)

# from threading import Thread

# thread = Thread(target=run)
# thread.start()
# =====================================
import uvicorn
from multiprocessing import Process

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start FastAPI using multiprocessing
process = Process(target=run)
process.start()

# Give the server a moment to start
import time
time.sleep(3)
```

---
### **4. Creating a Test Dummy Client**

To test data transfer between a client and the server, we'll use Python's `requests` library to create a dummy client.

First, let's install the necessary library:

```python
!pip install requests
```

Now, let's create a simple client to send a GET request to our server:

```python
import requests

response = requests.get("http://0.0.0.0:8000/")
print(response.json())
```

---
## **Conclusion**

With FastAPI, you can quickly set up server-side applications. Data transfer between the client and server can be effortlessly tested using Python's `requests` library.

In [ ]:
# Install
!pip install fastapi[all]
!pip install pyngrok
!pip install uvicorn

# Import
from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse, JSONResponse
from multiprocessing import Process, Queue
from pyngrok import ngrok
import time
import uvicorn

# Create a Queue for inter-process communication
log_queue = Queue()

# Set up ngrok tunnel to forward traffic to port 8000 of google colab device
ngrok.set_auth_token('2Xeavm8XuD0oCtFFSTrPQ3lczUj_6RrvYYLBxNgrwzLmeANYk')
public_url = ngrok.connect(addr="8000")
print(f"Public URL: {public_url}")

# Create FastAPI app
app = FastAPI()

# Handle get request to /
@app.get("/", response_class=HTMLResponse)
def read_root():
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>My Form</title>
        <style>
            /* Add your CSS styling here */
            body { font-family: Arial, sans-serif; }
            input { margin: 10px; }
        </style>
    </head>
    <body>
        <form action="/submit" method="post" accept-charset="UTF-8">
            <input type="text" name="input_field" placeholder="Enter something">
            <input type="submit" value="Submit">
        </form>
        <script>
            // Add your JavaScript here if needed
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content, media_type="text/html; charset=utf-8")

# Handle post request to /submit
@app.post("/submit")
async def submit(input_field: str = Form(...)):
    message = f"input_field: {input_field}"
    log_queue.put(message)  # Put the log message into the queue
    return JSONResponse(content={"received_input": input_field}, media_type="application/json; charset=utf-8")

# Start FastAPI
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Function to print log messages from the Queue
def print_log_messages(queue):
    while True:
        message = queue.get()  # Get the log message from the queue
        print(message)

# Using multiprocessing
run_process = Process(target=run)
run_process.start()
print_process = Process(target=print_log_messages, args=(log_queue,))
print_process.start()

# Give the server a moment to start
time.sleep(3)

# Block
blocking_main_proc = input()

# Kill
run_process.terminate()
print_process.terminate()

Public URL: NgrokTunnel: "https://ec37-35-199-13-31.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [895]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     39.125.21.116:0 - "GET / HTTP/1.1" 200 OK
INFO:     39.125.21.116:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
